In [9]:
import pandas as pd
import sqlite3
import random
from datetime import datetime, timedelta

In [10]:
DB_PATH = "/home/drake/scripts/sql_python_files/derp1.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

In [11]:
tquery = pd.read_sql_query("""
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
""", conn)
print(tquery)

DatabaseError: Execution failed on sql '
SELECT name
FROM sqlite_master
WHERE type='table'
ORDER BY name;
': database is locked

In [12]:
# Get table structure info
pragma_df = pd.read_sql_query("PRAGMA table_info(request_logs);", conn)
print(pragma_df.to_string(index=False))

DatabaseError: Execution failed on sql 'PRAGMA table_info(request_logs);': database is locked

In [ ]:
import sqlite3, random
from datetime import datetime, timedelta, timezone

DB_PATH = "/home/drake/scripts/sql_python_files/derp1.db"
conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

EVENTS = ["login_success", "login_failure", "password_reset", "locked_account"]
IPS = ["192.168.1.10", "203.0.113.55", "10.0.0.77", "198.51.100.23"]
AGENTS = ["Firefox", "Chrome", "Safari", "Edge"]

def random_time():
    start = datetime.now(timezone.utc) - timedelta(days=30)
    random_minutes = random.randint(0, 60*24*30)
    return (start + timedelta(minutes=random_minutes)).isoformat()

rows_to_insert = []
for _ in range(5000):
    user_id = random.randint(1, 200)   # assume ~200 users
    event = random.choice(EVENTS)
    ip = random.choice(IPS)
    agent = random.choice(AGENTS)
    ts = random_time()
    rows_to_insert.append((user_id, event, ip, agent, ts))

cur.executemany("""
INSERT INTO auth_events (user_id, event, ip, user_agent, ts)
VALUES (?, ?, ?, ?, ?)
""", rows_to_insert)

print("gj")

In [ ]:
query5 = pd.read_sql_query("""
SELECT COUNT(*)
FROM request_logs;
""", conn)

print(query5)

In [ ]:
query6 = pd.read_sql_query("""
SELECT env,COUNT(*)
FROM request_logs
WHERE env="prod"
GROUP BY env;
""", conn)
print(query6)

In [6]:
query7 = pd.read_sql_query("""
SELECT name
FROM sqlite_master
WHERE type='table'
""", conn)
print(query7)

DatabaseError: Execution failed on sql '
SELECT name
FROM sqlite_master
WHERE type='table'
': database is locked

In [ ]:
query8 = pd.read_sql_query("""
SELECT *
FROM auth_events
""", conn)
print(query8)

In [1]:
query9 = pd.read_sql_query("""
SELECT event, COUNT(*)
FROM auth_events
WHERE event="LOGIN_FAILURE or event="login_failure";
""", conn)
print(query9)

NameError: name 'pd' is not defined

In [72]:
query10= pd.read_sql_query("""
SELECT ip, event, COUNT(*) as count
FROM auth_events
GROUP BY ip
ORDER BY count;
""", conn)
print(query10)

                ip          event  count
0    198.51.100.23  LOGIN_SUCCESS      2
1     203.0.113.10  LOGIN_SUCCESS      2
2     203.0.113.77  LOGIN_SUCCESS      2
3     203.0.113.11  LOGIN_FAILURE      4
4    192.168.1.219         logout     53
..             ...            ...    ...
507  192.168.1.158  login_failure     99
508      10.0.0.55  login_failure    103
509      10.0.0.59  login_success    104
510     10.0.0.235  login_failure    106
511     10.0.0.103  login_failure    108

[512 rows x 3 columns]


In [77]:
query11 = pd.read_sql_query("""
SELECT distinct event
FROM auth_events;
""", conn)
print(query11)

            event
0   LOGIN_SUCCESS
1   LOGIN_FAILURE
2          logout
3  password_reset
4   login_failure
5   login_success


In [97]:
query12 = pd.read_sql_query("""
SELECT event, ip, count(*) as count
FROM auth_events
WHERE event="login_failure" or event="LOGIN_FAILURE"
GROUP BY ip
ORDER BY count DESC;
""", conn)

print(query12)

             event             ip  count
0    login_failure  192.168.1.181     32
1    login_failure  192.168.1.139     32
2    login_failure     10.0.0.143     31
3    login_failure     10.0.0.131     31
4    login_failure  192.168.1.172     30
..             ...            ...    ...
504  login_failure     10.0.0.134     10
505  login_failure  192.168.1.253      9
506  login_failure      10.0.0.90      9
507  login_failure   192.168.1.53      8
508  LOGIN_FAILURE   203.0.113.11      2

[509 rows x 3 columns]


In [99]:
query13 = pd.read_sql_query("""
SELECT *
FROM auth_events;
""", conn)

print(query13)

                                     ts  user_id           event  \
0      2025-08-30T23:30:30.146440+00:00        1   LOGIN_SUCCESS   
1      2025-08-30T23:35:30.146443+00:00        2   LOGIN_FAILURE   
2      2025-08-30T23:40:30.146445+00:00        2   LOGIN_SUCCESS   
3      2025-08-31T05:54:06.253543+00:00        2   LOGIN_SUCCESS   
4      2025-08-31T05:52:06.254342+00:00        3   LOGIN_SUCCESS   
...                                 ...      ...             ...   
40005        2025-06-23T20:12:51.225162      101  password_reset   
40006        2025-06-11T00:58:51.225165      107  password_reset   
40007        2025-06-29T10:36:51.225167      143          logout   
40008        2025-08-28T01:42:51.225169      111  password_reset   
40009        2025-06-23T21:35:51.225172      363          logout   

                  ip                                       user_agent  
0       203.0.113.10                                      Mozilla/5.0  
1       203.0.113.11                   

In [101]:
query14 = pd.read_sql_query("""
SELECT ip, user_id, event, plan, created_at
FROM auth_events 
JOIN users 
ON auth_events.user_id = users.id;
""", conn)
print(query14)

                ip  user_id           event        plan  \
0     203.0.113.10        1   LOGIN_SUCCESS        free   
1     203.0.113.11        2   LOGIN_FAILURE         pro   
2     203.0.113.11        2   LOGIN_SUCCESS         pro   
3    198.51.100.23        2   LOGIN_SUCCESS         pro   
4     203.0.113.77        3   LOGIN_SUCCESS  enterprise   
..             ...      ...             ...         ...   
254  192.168.1.162        2          logout         pro   
255  192.168.1.208        3  password_reset  enterprise   
256  192.168.1.250        1   login_success        free   
257   192.168.1.90        1   login_success        free   
258     10.0.0.226        1   login_success        free   

                           created_at  
0    2025-07-02T04:30:30.146091+00:00  
1    2025-08-24T04:30:30.146107+00:00  
2    2025-08-24T04:30:30.146107+00:00  
3    2025-08-24T04:30:30.146107+00:00  
4    2025-08-30T04:30:30.146110+00:00  
..                                ...  
254  2025-0

In [105]:
# I want to see number of login failures, for each ip, descending order, user id for those ips, and plan

query15 = pd.read_sql_query('''
SELECT ip, user_id, event, plan, count(*) as count
FROM auth_events
JOIN users
ON auth_events.user_id = users.id
WHERE event = "LOGIN_FAILURE" or event = "login_failure"
GROUP BY ip
ORDER BY count desc;
''', conn)

print(query15)

               ip  user_id          event        plan  count
0    203.0.113.11        2  LOGIN_FAILURE         pro      2
1   192.168.1.183        3  login_failure  enterprise      2
2   192.168.1.172        3  login_failure  enterprise      2
3       10.0.0.39        3  login_failure  enterprise      2
4      10.0.0.133        1  login_failure        free      2
..            ...      ...            ...         ...    ...
56     10.0.0.157        1  login_failure        free      1
57     10.0.0.151        1  login_failure        free      1
58     10.0.0.131        2  login_failure         pro      1
59      10.0.0.12        2  login_failure         pro      1
60     10.0.0.112        1  login_failure        free      1

[61 rows x 5 columns]
